## Problem Description

Design an L-shaped house with as much square footage (floor area) as possible, within the limitations imposed by city codes and certain aesthetic considerations. A rectangular second story will be built above the largest part of the first floor, extending from the back wall but only overlapping half of the front wing. The dimensions of the front wing are x and y, where x is the width. The dimensions of the back wing are a and b, where b is the width. The total width of the house is b.

***Limitations and Constraints***



1.   The foundation must encompass no more than 3000 square feet.
2.   The width of the front wing must be within one-third to one-half of the total house width.
3.   Leave at least 1,500 square feet in the inside corner of the L for a pool and patio.
4.   The house will sit on a 90 x 150-foot lot with 10 foot minimum setbacks on either side and 25 foot minimum setbacks front and back.
5.   The front and back wings must not be disproportionately sized; that is, the length of the back wing should be greater than half of the length of the front wing.

**Reference:**

*Borland, Eureka: The Solver, Borland International, Scotts Valley, California, 1987.*

*This problem was solved as part of NLP Assignment in the course Mathematical Optimization with GAMS and Pyomo (Python) on udemy. (https://www.udemy.com/course/mathematical-optimization-with-gams-and-pyomo-python/)*

## Solution

**Variables:**

$y \in \mathbb{R}^+$: Length of the front wing.

$x \in \mathbb{R}^+$: Width of the front wing.

$a \in \mathbb{R}^+$: Length of the back wing.

$b \in \mathbb{R}^+$: Width of the back wing.

$\text{A}_{0} = (a*b)+(x*y)$: Area of first floor.

$\text{A}_{1} = (x*(a+y/2))$: Area of second floor.

$\text{A} = \text{A}_{0} + \text{A}_{1}$: Total area.

**Objective Function:**
\begin{equation}
\text{Max}.  \text{A}
\tag{0}
\end{equation}

**Constraints:**

\begin{equation}
\text{A}_{0} \le 3000
\tag{1}
\end{equation}

\begin{equation}
b/3 \le x \le b/2
\tag{2}
\end{equation}

\begin{equation}
y*(b-x) \ge 1500
\tag{3}
\end{equation}

\begin{equation}
y+a \le 100
\tag{4}
\end{equation}

\begin{equation}
b \le 70
\tag{5}
\end{equation}

\begin{equation}
a \ge y/2
\tag{6}
\end{equation}

In [1]:
!pip install pyomo
!apt-get install -y -qq glpk-utils
import pyomo.environ as pyomo

!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     |████████████████████████████████| 9.2 MB 5.5 MB/s 
     |████████████████████████████████| 49 kB 6.9 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up 

In [2]:
# Declare Model
model = pyomo.ConcreteModel()

# Declare Variables
model.y = pyomo.Var(domain=pyomo.PositiveReals)
model.a = pyomo.Var(domain=pyomo.PositiveReals)
model.b = pyomo.Var(domain=pyomo.PositiveReals)
model.x = pyomo.Var(domain=pyomo.PositiveReals)

# Define Constraints
def rule1(model):
  return (model.a*model.b) + (model.x*model.y) <= 3000
model.equ1 = pyomo.Constraint(rule = rule1, doc = 'Base area constraint')

def rule2a(model):
  return model.x >= model.b/3
model.equ2a = pyomo.Constraint(rule = rule2a, doc = ' x lower bound')

def rule2b(model):
  return model.x <= model.b/2
model.equ2b = pyomo.Constraint(rule = rule2b, doc = 'x upper bound')

def rule3(model):
  return model.y*(model.b-model.x) >= 1500
model.equ3 = pyomo.Constraint(rule = rule3, doc = 'Constraint for pool and patio')

def rule4(model):
  return (model.y+model.a) <= 100
model.equ4 = pyomo.Constraint(rule = rule4, doc = 'Sides constraint')

def rule5(model):
  return model.b <= 70
model.equ5 = pyomo.Constraint(rule = rule5, doc = 'b constraint')

def rule6(model):
  return model.a >= model.y/2
model.equ6 = pyomo.Constraint(rule = rule6, doc = 'a constraint')

# Define Objective func
model.obj = pyomo.Objective(expr = ((model.a*model.b) + (model.x*model.y)) + (model.x*(model.a+(model.y/2))), sense = pyomo.maximize)

# Solver Function
results = pyomo.SolverFactory('ipopt',executable = '/content/ipopt').solve(model)

results.write()
print('\n Results \n')
print('Max area:', model.obj(), 'sq feet')
print('a:',model.a(), 'feet')
print('b:',model.b(), 'feet')
print('x:',model.x(), 'feet')
print('y:',model.y(), 'feet')

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 4
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.08557438850402832
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

 Resul